<a href="https://colab.research.google.com/github/ryan-saloma/teaching-python/blob/main/Project_1_(Taylor_Swift_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Libraries and Data

In [1]:
import pandas as pd

data = pd.read_csv('taylor_swift_spotify.csv')

In [42]:
# Explore the data

# Look at the first few rows
data.head()

# Look at the column names, number of rows, and data types
# Check for missing values
# data.info()

# Generate basic summary stats
# data.describe()

# Check for duplicates
# data['name'].value_counts()

# Attempt to separate originals from later releases
mask = data['album'].str.contains('Version', case=False, na=False) | \
       data['album'].str.contains('Edition', case=False, na=False) | \
       data['album'].str.contains('Live', case=False, na=False) | \
       data['album'].str.contains('Tour', case=False, na=False) | \
       data['album'].eq('THE TORTURED POETS DEPARTMENT')
versions_df = data[mask]
originals_df = data[~mask]

# Inspect album names
originals_df['album'].unique()


array(['THE TORTURED POETS DEPARTMENT: THE ANTHOLOGY', 'Midnights',
       'evermore', 'folklore', 'Lover', 'reputation', '1989', 'Red',
       'Speak Now', 'Fearless', 'Taylor Swift'], dtype=object)